<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_extra_3__tanie_loty.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -qU langchain_community crewai crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.0/192.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━

Kod instaluje kilka pakietów Pythona używając menedżera pakietów pip:

- langchain_community to biblioteka zawierająca komponenty społecznościowe do budowania aplikacji z użyciem modeli językowych
- crewai to framework do tworzenia zespołów agentów AI, które mogą współpracować przy wykonywaniu zadań
- crewai_tools to zestaw narzędzi rozszerzających możliwości crewai

Flaga -q oznacza "quiet mode" - instalacja będzie przebiegać bez wyświetlania szczegółowych komunikatów.

Flaga -U wymusza aktualizację pakietów do najnowszych dostępnych wersji, nawet jeśli są już zainstalowane w systemie.

Ten jeden wiersz przygotowuje środowisko do pracy z agentami AI opartymi na dużych modelach językowych.

In [ ]:
# Standard library imports
import os
import warnings

# Third-party imports
from crewai import Agent, Task, Crew
from crewai.process import Process
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool
from dotenv import load_dotenv, find_dotenv
from langchain_community.llms import Ollama
from langchain_community.utilities import ApifyWrapper
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI

from google.colab import userdata

import json
# Suppress warnings
warnings.filterwarnings('ignore')


from pprint import pp as ppr


Ten kod importuje szereg bibliotek i narzędzi niezbędnych do pracy z AI i przetwarzania danych:

Ze standardowej biblioteki Pythona:
- os - do operacji systemowych
- warnings - do zarządzania ostrzeżeniami (które są tutaj wyłączone)

Z zewnętrznych bibliotek:
- crewai - importuje główne komponenty do tworzenia zespołów AI:
  - Agent - do tworzenia pojedynczych agentów AI
  - Task - do definiowania zadań
  - Crew - do łączenia agentów w zespoły
  - Process - do zarządzania procesami

- crewai_tools dostarcza specjalistyczne narzędzia:
  - SerperDevTool - do wyszukiwania w internecie
  - ScrapeWebsiteTool - do pobierania danych ze stron
  - WebsiteSearchTool - do przeszukiwania stron

Dodatkowo:
- dotenv - do ładowania zmiennych środowiskowych
- langchain_community.llms.Ollama - do integracji z modelem Ollama
- langchain_openai.ChatOpenAI - do komunikacji z OpenAI
- google.colab.userdata - do obsługi danych użytkownika w Google Colab
- json - do pracy z formatem JSON
- pprint (jako ppr) - do ładnego formatowania wydruków

Kod konfiguruje też środowisko wyłączając ostrzeżenia przez `warnings.filterwarnings('ignore')`.

In [ ]:
class CFG:
    model = 'gpt-4o-mini'
    temp = 0

In [ ]:
os.environ["OPENAI_API_KEY"] = userdata.get('openaivision')
os.environ["SERPER_API_KEY"] = userdata.get('serper')

# Agenci

In [ ]:
llm = ChatOpenAI(model= CFG.model, temperature = CFG.temp)

Ten wiersz kodu tworzy instancję modelu językowego ChatOpenAI z dwoma parametrami pobranymi z obiektu konfiguracyjnego CFG:

- model - określa konkretny model ChatGPT, który ma być używany (nazwa modelu jest zapisana w CFG.model)
- temperature - ustawia parametr losowości odpowiedzi (wartość z CFG.temp)

Temperature to parametr w zakresie 0-1:
- Niskie wartości (bliskie 0) dają bardziej przewidywalne, spójne odpowiedzi
- Wysokie wartości (bliskie 1) zwiększają kreatywność i różnorodność odpowiedzi

Utworzona instancja llm będzie używana do generowania tekstu i komunikacji z API OpenAI.

In [ ]:
# load tools
serper_search_tool = SerperDevTool()
scrap_tool = ScrapeWebsiteTool()
website_search_tool = WebsiteSearchTool()

# define the tools to be used by the Agents
tools=[serper_search_tool, scrap_tool, website_search_tool]


Te linie kodu przygotowują zestaw narzędzi dla agentów AI:

Tworzą trzy narzędzia:
- serper_search_tool - narzędzie do wyszukiwania w internecie przez API Serper
- scrap_tool - narzędzie do wyciągania treści ze stron internetowych
- website_search_tool - narzędzie do przeszukiwania konkretnych stron

Następnie te trzy narzędzia są łączone w listę tools, która będzie przekazana agentom do wykorzystania podczas wykonywania zadań.

Każde z tych narzędzi rozszerza możliwości agentów w inny sposób:
- pierwsze pozwala im znajdować informacje w sieci
- drugie umożliwia pobieranie zawartości stron
- trzecie pomaga w przeszukiwaniu konkretnych witryn

In [ ]:
# agent definitions
travel_expert = Agent(
  role='Specjalista ds. lotów',
  goal='Efektywnie wyszukiwać i tworzyć kompleksową listę dostępnych połączeń lotniczych między wskazanymi lokalizacjami,\
  przestrzegając określonych kryteriów, takich jak daty wylotu/przylotu, liczba pasażerów, preferowane linie lotnicze\
  i inne szczególne wymagania. Zapewnić, że zebrane informacje są dokładne, aktualne i zawierają kluczowe szczegóły\
  jak ceny, czas trwania lotu, liczba przesiadek i dostępność miejsc.',
  backstory='Jesteś wyspecjalizowanym ekspertem ds. rezerwacji lotów z wyczuleniem na szczegóły i zamiłowaniem do planowania \
optymalnych tras podróży. Dzięki wieloletniemu doświadczeniu w branży lotniczej i rozległej wiedzy o globalnych\
przewoźnikach oraz połączeniach regionalnych, doskonale poruszasz się po systemach rezerwacyjnych i rozkładach lotów.\
Twoja specjalizacja polega na sprawnym analizowaniu dostępnych połączeń w celu identyfikacji najbardziej korzystnych\
opcji dla podróżnych, uwzględniając takie czynniki jak czas lotu, cena, liczba przesiadek i reputacja linii lotniczych.',
  verbose=True,
  allow_delegation=False,
  llm=llm,
  tools=tools,
)


hotel_reviewer = Agent(
  role='Analityk Jakości Hoteli',
    goal = """
    Dokładnie oceniać i porównywać opcje lotnicze, aby dostarczać szczegółowych, bezstronnych analiz pomagających
    podróżnym w podejmowaniu świadomych decyzji. Analizować punktualność lotów, komfort podróży, jakość obsługi,
    efektywność przesiadek i stosunek jakości do ceny każdego połączenia, aby tworzyć kompleksowe oceny podkreślające
    mocne strony, słabości i specyficzne cechy poszczególnych opcji.""",
  backstory = """
  Jesteś doświadczonym analitykiem branży lotniczej z ponad dziesięcioletnim doświadczeniem w ocenie jakości połączeń
  i usług linii lotniczych. Twoje doświadczenie obejmuje pracę jako audytor jakości dla międzynarodowych linii lotniczych,
  dziennikarz specjalizujący się w tematyce lotniczej oraz konsultant systemów oceny przewoźników. To różnorodne
  doświadczenie rozwinęło Twoją zdolność do oceny połączeń z wielu perspektyw, uwzględniając zarówno obiektywne
  wskaźniki jak punktualność i niezawodność, jak i subiektywne doświadczenia pasażerów. Masz szczególny talent do
  identyfikowania optymalnych połączeń i dostrzegania potencjalnych problemów, które mogą wpłynąć na komfort podróży.""",
  verbose=True,
  allow_delegation=False,
  llm=llm,
  tools=tools,
)

Ten kod tworzy dwóch specjalizowanych agentów AI do obsługi zadań związanych z hotelami:

travel_expert to agent specjalizujący się w wyszukiwaniu hoteli:
- jego rola to "Specjalista ds. Hoteli"
- ma za zadanie wyszukiwać opcje hotelowe według określonych kryteriów
- posiada "backstory" jako doświadczony concierge z wiedzą o branży
- ma dostęp do wcześniej zdefiniowanych narzędzi (tools)
- pracuje z modelem językowym (llm)
- ma włączone szczegółowe logowanie (verbose=True)
- nie może delegować zadań (allow_delegation=False)

hotel_reviewer to agent oceniający hotele:
- pełni rolę "Analityka Jakości Hoteli"
- jego zadaniem jest szczegółowa analiza i porównywanie opcji hotelowych
- ma "backstory" jako ekspert z 10-letnim doświadczeniem w branży
- używa tych samych narzędzi i modelu co travel_expert
- również ma włączone logowanie i wyłączoną delegację zadań

Obaj agenci mają szczegółowo określone cele i historie, które wpływają na sposób, w jaki będą podchodzić do powierzonych im zadań.

In [ ]:
# tasks

hotel_search_task = Task(
      description = "Wyszukaj 5 połączeń z {location1} do {location2} w terminie {data1} do {data2} w cenie max {cenamax} PLN. Tylko połączenia bezpośrednie,\
        podaj numery lotów.",
      agent = travel_expert,
      expected_output = "Wszystkie szczegóły dotyczące konkretnie wybranego połączenia."
)



hotel_review_task = Task(
  description="Na podstawie dostarczonych rekomendacji, wybierz najlepsze opcje w oparciu o oceny, recenzje i dostępne udogodnienia. \
  Weź pod uwagę, że budżet wynosi {cenamax} PLN. Postaraj się znaleźć połączenie w granicach tego budżetu.",
  expected_output="Wszystkie szczegóły dotyczące konkretnie wybranego połączenia, w tym cena, URL.",
  agent = hotel_reviewer,
)

Ten kod definiuje dwa zadania dla wcześniej utworzonych agentów:

hotel_search_task to zadanie wyszukiwania hoteli:
- przypisane do agenta travel_expert
- opis zawiera parametry, które będą uzupełniane przy uruchomieniu:
  - {location} - lokalizacja
  - {number_of_people} - liczba osób
  - {check_in} - data zameldowania
  - {check_out} - data wymeldowania
- oczekuje zwrotu szczegółowych informacji o znalezionych opcjach zakwaterowania

hotel_review_task to zadanie analizy i wyboru najlepszych opcji:
- przypisane do agenta hotel_reviewer
- uwzględnia parametr {budget} określający limit cenowy w PLN
- ma wybrać najlepsze opcje na podstawie:
  - ocen
  - recenzji
  - dostępnych udogodnień
- oczekuje szczegółowego raportu zawierającego:
  - cenę
  - link do oferty
  - dostępne zdjęcia

Te zadania tworzą dwuetapowy proces: najpierw wyszukanie opcji, potem ich szczegółowa analiza i wybór najlepszych propozycji.

In [ ]:
# Create the Crew
travel_agent_crew = Crew(
    agents=[travel_expert, hotel_reviewer],
    tasks=[hotel_search_task, hotel_review_task],
    verbose=True
)

event_criteria = {
    'location1': 'Gdańsk, Poland',
    'location2': 'Berlin, Germany',
    'data1': '22 grudnia 2024',
    'data2': '27 grudnia 2024',
    'cenamax': 2000
}

result = travel_agent_crew.kickoff(inputs=event_criteria)

# Agent: Specjalista ds. lotów
## Task: Wyszukaj 5 połączeń z Gdańsk, Poland do Berlin, Germany w terminie 22 grudnia 2024 do 27 grudnia 2024 w cenie max 2000 PLN. Tylko połączenia bezpośrednie,        podaj numery lotów.


# Agent: Specjalista ds. lotów
## Thought: Muszę wyszukać połączenia lotnicze z Gdańska do Berlina w określonym terminie, uwzględniając wszystkie wymagane kryteria. Zacznę od przeszukania internetu w celu znalezienia dostępnych połączeń.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"flights from Gda\\u0144sk to Berlin December 22 2024 to December 27 2024 direct flights under 2000 PLN\"}"
## Tool Output: 

Search results: Title: Find Cheap Flights from Gdańsk to Berlin - Google
Link: https://www.google.com/travel/flights/flights-from-gdansk-to-berlin.html
Snippet: The cheapest round-trip flight from Gdańsk to Berlin starts at $157 from Thu, Jan 9 to Sun, Jan 12. The cheapest one-way flight starts at $91 and departs on Wed ...
---
Title: Che

In [ ]:
ppr(result.raw)